In [37]:
import pandas as pd
train=pd.read_csv('../input/song-popularity-prediction/train.csv')

In [38]:
train.head()

In [3]:
train.info()

In [5]:
#applying mean to replace null in duration of the song
import numpy as np
dur_mean=np.mean(train['song_duration_ms'])
#train['song_duration_ms'].fillna(dur_mean,inplace=True)

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt
train.hist(figsize=(12,10))
plt.show()

EDA of Each Feature

In [4]:
# song_duration_ms plot
sns.set_theme()
ax=sns.displot(data=train,x='song_duration_ms',kind='kde',color='red',height=4,aspect=3) 
ax=sns.displot(data=train,x='song_duration_ms',kind='kde',height=4,aspect=3,hue='song_popularity') #Kernel Distribution Estimation

Hence from the above data we see that the songs in the range of 3mins are most popular

In [7]:
# acousticness plot
import numpy as np
ax=sns.displot(data=train,x='acousticness',kind='kde',color='orange',height=5,aspect=3)
ax=sns.displot(data=train,x=np.log(train['acousticness']),kind='kde',color='orange',height=5,aspect=3)
ax=sns.displot(data=train,x='acousticness',kind='kde',height=5,aspect=3,hue='song_popularity') 

Taking a log doesnt help much in improving the trend, so as of now we dont do a log transformation for this column

In [ ]:
# danceability plot
ax=sns.displot(data=train,x='danceability',kind='kde',color='green',height=4,aspect=3)
ax=sns.displot(data=train,x='danceability',kind='kde',height=4,aspect=3,hue='song_popularity')

In [ ]:
# energy plot
ax=sns.displot(data=train,x='energy',kind='kde',color='purple',height=4,aspect=3) 
ax=sns.displot(data=train,x='energy',kind='kde',height=4,aspect=3,hue='song_popularity') 

In [ ]:
# instrumentalness plot

ax=sns.displot(data=train,x='instrumentalness',kind='kde',color='brown',height=4,aspect=3) 
ax=sns.displot(data=train,x=np.log(train['instrumentalness']),kind='kde',color='brown',height=4,aspect=3) 
ax=sns.displot(data=train,x='instrumentalness',kind='kde',height=4,aspect=3,hue='song_popularity') 

Using the log of the feature looks a reasonable outcome

In [ ]:
# key plot
sns.displot(data=train,x='key',color='darkblue',height=4,aspect=3) 
ax=sns.displot(data=train,x='key',height=4,aspect=3,hue='song_popularity',multiple="stack") 

In [ ]:
# liveness plot
ax=sns.displot(data=train,x='liveness',kind='kde',color='Magenta',height=4,aspect=3) 
ax=sns.displot(data=train,x=np.log(train['liveness']),kind='kde',color='Magenta',height=4,aspect=3)
ax=sns.displot(data=train,x='liveness',kind='kde',height=4,aspect=4,hue='song_popularity') 

Using the log of this feature as we get a good distribution using the latter

In [ ]:
# loudness plot
ax=sns.displot(data=train,x='loudness',kind='kde',color='Crimson',height=4,aspect=3) 
ax=sns.displot(data=train,x='loudness',kind='kde',height=4,aspect=4,hue='song_popularity') 

In [ ]:
#audio mode
sns.displot(data=train,x='audio_mode',color='cyan',height=4,aspect=3) 
ax=sns.displot(data=train,x='audio_mode',height=4,aspect=3,hue='song_popularity',multiple="stack") 

In [ ]:
#Speechiness 
ax=sns.displot(data=train,x='speechiness',kind='kde',color='lightGreen',height=4,aspect=3)
ax=sns.displot(data=train,x=np.log(train['speechiness']),kind='kde',color='lightGreen',height=4,aspect=3)
ax=sns.displot(data=train,x='speechiness',kind='kde',height=4,aspect=4,hue='song_popularity') 

Using log of this feature

In [ ]:
#tempo
ax=sns.displot(data=train,x='tempo',kind='kde',color='Black',height=4,aspect=3) 
ax=sns.displot(data=train,x='tempo',kind='kde',height=4,aspect=4,hue='song_popularity') 

In [ ]:
#Time signature
sns.displot(data=train,x='time_signature',color='DarkOrange',height=4,aspect=3) 
ax=sns.displot(data=train,x='time_signature',height=4,aspect=3,hue='song_popularity',multiple="stack")

In [ ]:
#Audio Valence
ax=sns.displot(data=train,x='audio_valence',kind='kde',color='tan',height=4,aspect=3) 
ax=sns.displot(data=train,x='audio_valence',kind='kde',height=4,aspect=4,hue='song_popularity') 

In [ ]:
#Song popularity
sns.displot(data=train,x='song_popularity',color='maroon',height=4,aspect=3) 

There is a class imbalance, though not hugely imbalanced. But there is some imbalance between the classes

In [7]:
train.info()

In [39]:
#converting the songs in mins
train['song_duration_ms']=train['song_duration_ms']/(1000*60)

In [40]:
train.rename(columns={'song_duration_ms':'song_duration_mins'},inplace=True)

In [41]:
#Tranforming columns intrumentalness, liveness, speechiness
train['log_instrumentalness']=train['instrumentalness'].apply(lambda x: np.log(x))
#train['log_liveness']=train['liveness'].apply(lambda x: np.log(x))
#train['log_speechiness']=train['speechiness'].apply(lambda x: np.log(x))

In [11]:
y_train=train['song_popularity']
X_train=pd.DataFrame(train)

In [12]:
X_train

In [ ]:
X_train.drop(['id','instrumentalness','song_popularity'],axis=1,inplace=True)

In [ ]:
X_train

In [ ]:
import xgboost as xgb

xgb_clf=xgb.XGBClassifier(random_state=42,n_estimators=50,max_depth=10,learning_rate=0.38)
xgb_clf.fit(X_train[features_xgb],y_train)

In [36]:
sub=test[['id']]
sub['song_popularity']=smote_cv.predict(X_test)
sub['song_popularity'].value_counts()

In [ ]:
sub.to_csv('submission.csv')

In [ ]:
train.head()

In [23]:
#Checking correlation of the columns - with the null values
fig, ax = plt.subplots(figsize=(20,10))
ax=sns.heatmap(X_train.corr(),annot=True,linewidths=.5)

In [ ]:
features_xgb=['acousticness','energy','loudness','log_instrumentalness']  

In [ ]:
train.describe()

In [ ]:
train['key'].value_counts()

In [13]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.nan,strategy='mean')
cols_to_be_imputed_with_mean=['song_duration_mins','acousticness','danceability','energy','loudness','log_instrumentalness']
train[cols_to_be_imputed_with_mean]=imputer.fit_transform(train[cols_to_be_imputed_with_mean])

In [ ]:
imputer.statistics_

In [14]:
#Imputing key with mode
impute_mode=SimpleImputer(missing_values=np.nan,strategy='most_frequent')
train['key']=impute_mode.fit_transform(train['key'].to_numpy().reshape(-1,1))

In [15]:
#Median imputer
impute_med=SimpleImputer(missing_values=np.nan,strategy='median')
cols_to_be_imputed_with_median=['instrumentalness','liveness']
train[cols_to_be_imputed_with_median]=impute_med.fit_transform(train[cols_to_be_imputed_with_median])

In [16]:
train.info()

In [17]:
fig, ax = plt.subplots(figsize=(20,10))
ax=sns.heatmap(train.corr(),annot=True,linewidths=.5)

In [ ]:
#Sampling data
#count_of_non_popular_song=len(train[train['song_popularity']==0])
#popular_songs=train[train['song_popularity']==1]
#train_oversampled=popular_songs.sample(count_of_non_popular_song,replace=True)
#X_train_sampled=pd.concat([train_oversampled,train[train['song_popularity']==0]])

In [ ]:
train

In [42]:
#Scaling the features 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
pipeline=Pipeline([('std_scaler',StandardScaler())])

In [43]:
#Using columns 'acousticness','energy','loudness','audio_valence','danceability','log_instrumentalness' as per the insight received from the above heatmap
#features_set1=['acousticness','energy','loudness','audio_valence','log_intrumentalness']
y_train=train['song_popularity']
X_train=pd.DataFrame(train)
X_train.drop(['instrumentalness','song_popularity','id'],axis=1,inplace=True)

In [44]:
pipeline.fit_transform(X_train)

In [45]:
from sklearn.neighbors import KNeighborsClassifier

knn=KNeighborsClassifier()
knn.get_params()

In [21]:
X_train

In [22]:
sns.scatterplot(data=train, x="energy", y="loudness", hue="song_popularity")

In [18]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
over = SMOTE()
under = RandomUnderSampler()
steps = [('o', over), ('u', under),('model', model)]
pipeline_smote = Pipeline(steps=steps)

In [24]:
from sklearn.model_selection import GridSearchCV
params={'model__n_estimators':[50,100,150],'model__max_features':[4,6]}

smote_cv=GridSearchCV(estimator=pipeline_smote,param_grid=params,cv=5,scoring='roc_auc',verbose=10)
smote_cv.fit(X_train,y_train)

In [27]:
best_smote=smote_cv.best_estimator_.fit(X_train,y_train)

In [95]:
smote_cv.get_params().keys()

In [89]:
model.get_params()

In [45]:
X_train,y_train=pipeline_smote.fit_resample(X_train,y_train)

In [51]:
(y_train==0).count()

In [ ]:
svc_clf.get_params()

In [ ]:
#Creating a validation set
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

svc_clf=LinearSVC(random_state=42)
param_svc={'C':[5,10],'tol':[0.005,0.05]}

svc_grid=GridSearchCV(svc_clf,param_svc,verbose=10,cv=5,scoring='roc_auc')
svc_grid.fit(X_train,y_train)

In [ ]:
svc_grid.best_estimator_

In [ ]:
#Building a model with best estimator
svc_final=svc_grid.best_estimator_
svc_final.fit(X_train,y_train)

In [29]:
test=pd.read_csv('../input/song-popularity-prediction/test.csv')

In [30]:
#Tweaking the time data 
test['song_duration_ms']=test['song_duration_ms']/1000*60
test.rename(columns={'song_duration_ms':'song_duration_mins'},inplace=True)

In [31]:
#Creating log of feature instrumentalness
test['log_instrumentalness']=test['instrumentalness'].apply(lambda x: np.log(x))
#test['log_liveness']=test['liveness'].apply(lambda x: np.log(x))
#test['log_speechiness']=test['speechiness'].apply(lambda x: np.log(x))

In [33]:
X_test=pd.DataFrame(test)
X_test

In [34]:
X_test.drop(['id','instrumentalness'],inplace=True,axis=1)

In [35]:
X_test

In [32]:
#imputing the test data
test[cols_to_be_imputed_with_mean]=imputer.transform(test[cols_to_be_imputed_with_mean])
test['key']=impute_mode.transform(test['key'].to_numpy().reshape(-1,1))
test[cols_to_be_imputed_with_median]=impute_med.transform(test[cols_to_be_imputed_with_median])

In [ ]:
X_test=pipeline.transform(test)

In [ ]:
sub['song_popularity']=svc_final.predict(test)

In [ ]:
sub['song_popularity'].value_counts()

In [ ]:
ens1.get_params()

In [ ]:
#Running RandomForestRegressor on the whole dataset
X_train=train
X_train=pipeline.fit_transform(X_train)

In [ ]:
X_train

In [5]:
from sklearn.ensemble import RandomForestClassifier

ens1=RandomForestClassifier(random_state=42)
param_grid={'n_estimators':[50,100,200],'max_features': [4,6,8]}

#grid_ens=GridSearchCV(ens1,param_grid,verbose=10,scoring='roc_auc',cv=10)
#grid_ens.fit(X_train,y_train)

In [6]:
ens1.get_params()

In [ ]:
grid_ens.best_estimator_

In [ ]:
ens_final=RandomForestClassifier(max_features=6, random_state=42,n_estimators=50)
ens_final.fit(X_train,y_train)

In [ ]:
sub['song_popularity']=ens_final.predict(X_test)

In [ ]:
sub['song_popularity'].value_counts()

In [ ]:
y_val_pred=ens1.predict(X_val)

In [ ]:
#accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_val,y_val_pred)

In [ ]:
test.info()

In [ ]:
test.drop(['instrumentalness','liveness','speechiness'],axis=1,inplace=True)

In [ ]:
test

In [ ]:
X_test=pipeline.transform(test)

In [ ]:
X_test

In [ ]:
sub=test[['id']]

In [ ]:
sub['song_popularity']=ens_final.predict(X_test)

In [ ]:
sub['song_popularity'].value_counts()   #50

In [ ]:
sub.to_csv('submission.csv')

In [ ]:
y_train=X_train_sampled['song_popularity']
#X_train_sampled.drop(['song_popularity'],axis=1,inplace=True)

In [ ]:
#X_train_sampled.drop(['instrumentalness','liveness','speechiness'],axis=1,inplace=True)

In [ ]:
#X_train_sampled.head()

In [ ]:
#from sklearn.ensemble import RandomForestClassifier

#ens1=RandomForestClassifier(random_state=42)
#param_grid={'n_estimators':[50,100,200],'max_features': [4,6,8]}

#grid_ens=GridSearchCV(ens1,param_grid,verbose=10,scoring='roc_auc',cv=5)
#grid_ens.fit(X_train_sampled,y_train)

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp_clf = MLPClassifier(random_state=42,max_iter=300)
mlp_clf.fit(X_train,y_train)

In [ ]:
sub['song_popularity']=final_tree.predict(X_test)

In [ ]:
sub['song_popularity'].value_counts()

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

ens1=ExtraTreesClassifier(random_state=42,class_weight='balanced')
param_grid={'n_estimators':[50,100,200],'max_features': [4,6,8]}

grid_ens=GridSearchCV(ens1,param_grid,verbose=10,scoring='roc_auc',cv=5)
grid_ens.fit(X_train,y_train)

In [ ]:
grid_ens.best_estimator_

In [ ]:
final_tree=ExtraTreesClassifier(max_features=6, n_estimators=50,random_state=42)
final_tree.fit(X_train,y_train)

In [ ]:
#Stacking Ensemble
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
random_forest_clf = RandomForestClassifier(n_estimators=50, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=50, random_state=42)
mlp_clf = MLPClassifier(random_state=42,max_iter=300)

In [ ]:
#Splitting the training data in train and val
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2)

In [ ]:
estimators = [random_forest_clf, extra_trees_clf, mlp_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train[features_xgb], y_train)

In [ ]:
#Predict the val output using each estimator and storing the prediction to fit a stack on the validation set
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [ ]:
rnf_blender=RandomForestClassifier(n_estimators=200,oob_score=True, random_state=42)
rnf_blender.fit(X_val_predictions,y_val)

In [ ]:
X_test_predictions = np.empty((len(test), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(test)

In [ ]:
sub['song_popularity']=rnf_blender.predict(X_test_predictions)

In [ ]:
sub['song_popularity'].value_counts()